# Cleaning item-specific data

In [11]:
import pandas as pd
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'adria'
password = '00000'
ip = '35.187.114.125'
database = 'vimet'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [12]:
items = pd.read_sql('items', con = engine)

In [13]:
items.head()

,index,Name,Id,Lineitem quantity,Lineitem name,Lineitem price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Lineitem compare at price
0,0,#1247,8.925310e+11,1,Cesta de temporada (1/2 pensión) - 2,59.0,NaN,1,0,fulfilled,NaN
1,1,#1246,8.918520e+11,1,Cesta familiar (1/2 pensión) - 3,79.0,NaN,1,0,pending,NaN
2,2,#1246,NaN,1,Desayuno salado - 2,15.0,NaN,1,0,pending,NaN
3,3,#1245,8.917250e+11,1,FAM - QA - 17 - 2,32.5,NaN,1,0,fulfilled,NaN
4,4,#1244,8.910100e+11,1,Cesta de temporada sin pescado (1/2 pensión) - 2,45.0,NaN,1,0,fulfilled,NaN


We'll drop the index column as it is redundant and the 'id' (which corresponds to the order and it's redundant with 'Name'

In [14]:
items = items.drop(columns=['index', 'Id'])

In [15]:
items.head()

,Name,Lineitem quantity,Lineitem name,Lineitem price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Lineitem compare at price
0,#1247,1,Cesta de temporada (1/2 pensión) - 2,59.0,NaN,1,0,fulfilled,NaN
1,#1246,1,Cesta familiar (1/2 pensión) - 3,79.0,NaN,1,0,pending,NaN
2,#1246,1,Desayuno salado - 2,15.0,NaN,1,0,pending,NaN
3,#1245,1,FAM - QA - 17 - 2,32.5,NaN,1,0,fulfilled,NaN
4,#1244,1,Cesta de temporada sin pescado (1/2 pensión) - 2,45.0,NaN,1,0,fulfilled,NaN


## NaN values

In [16]:
items.isna().sum()

Name                              0
Lineitem quantity                 0
Lineitem name                     0
Lineitem price                    0
Lineitem sku                     83
Lineitem requires shipping        0
Lineitem taxable                  0
Lineitem fulfillment status       0
Lineitem compare at price      2396
dtype: int64

In [18]:
items.shape

(2419, 9)

In [20]:
items['Lineitem compare at price'].value_counts()

0.0    23
Name: Lineitem compare at price, dtype: int64

We will drop 'Lineitem compare at price' as it is either 0 or NaN

In [21]:
items = items.drop(columns='Lineitem compare at price')

        # Preguntar a Ramon que es Lineitem sku

In [26]:
items = items.drop(columns='Lineitem sku')

## Variable types

In [28]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2419 entries, 0 to 2418
Data columns (total 7 columns):
Name                           2419 non-null object
Lineitem quantity              2419 non-null int64
Lineitem name                  2419 non-null object
Lineitem price                 2419 non-null float64
Lineitem requires shipping     2419 non-null int64
Lineitem taxable               2419 non-null int64
Lineitem fulfillment status    2419 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 132.4+ KB


Types seem ok

## Feature values

In [29]:
items.columns

Index(['Name', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem requires shipping', 'Lineitem taxable',
       'Lineitem fulfillment status'],
      dtype='object')

#### Quantity -- OK

In [30]:
items['Lineitem quantity'].value_counts()

1     2106
2      210
3       54
4       32
5       10
6        5
7        1
10       1
Name: Lineitem quantity, dtype: int64

Looks good

#### Lineitem name'

In [32]:
items['Lineitem name'].value_counts()

Calabacín (500 grs.)                                        38
Garbanzos cocidos (500 grs.)                                31
Pechuga de pollo fileteada (350 grs.)                       28
Lenteja cocida (500 grs.) - Pequeña (pardina)               26
Hamburguesa de cebolla queso y huevo (2 uds.)               25
Patata - Monalisa (1 Kg.) / 1.20€                           23
Naranjas zumo (2 kg.)                                       22
Nectarina (500 grs.)                                        21
Aguacate (2 unidades)                                       20
Puerro (1 Kg. = 2/3 uds.)                                   20
Cerezas Extra (250 grs.)                                    20
Tomate Monterosa (500 grs.)                                 20
Brocolí (1 ud.)                                             20
Plátano Canarias (3/4 unidades) - Verde                     19
Pimientos - Rojo (500 grs. 1/2 uds.) / 1.75€                19
Mandarina - 500 grs.                                   

#### 'Lineitem price' --OK

Looks good

#### 'Lineitem requires shipping' -- OK

In [34]:
items['Lineitem requires shipping'].value_counts()

1    2402
0      17
Name: Lineitem requires shipping, dtype: int64

####  'Lineitem taxable' -- OK

In [35]:
items['Lineitem taxable'].value_counts()

0    2411
1       8
Name: Lineitem taxable, dtype: int64

#### 'Lineitem fulfillment status' -- OK

In [36]:
items['Lineitem fulfillment status'].value_counts()

fulfilled    2318
pending       101
Name: Lineitem fulfillment status, dtype: int64

# Load data

In [37]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'adria'
password = '00000'
ip = '35.187.114.125'
database = 'vimet'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [38]:
items.to_sql('items', con = engine, if_exists='replace')